---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import re

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

### The document

In [3]:
with open('university_towns.txt', 'r') as f:
    uni = f.read()
print(uni[:497])

Alabama[edit]
Auburn (Auburn University)[1]
Florence (University of North Alabama)
Jacksonville (Jacksonville State University)[2]
Livingston (University of West Alabama)[2]
Montevallo (University of Montevallo)[2]
Troy (Troy University)[2]
Tuscaloosa (University of Alabama, Stillman College, Shelton State)[3][4]
Tuskegee (Tuskegee University)[5]
Alaska[edit]
Fairbanks (University of Alaska Fairbanks)[2]
Arizona[edit]
Flagstaff (Northern Arizona University)[6]
Tempe (Arizona State University)


In [4]:
def clean_text(file):
    uni = file
    states, dct, count, count2 = [], {}, 0, 1
    for line in uni:
        if re.search('^.*\[(.*)\]$',line) != None:
            test = re.findall('^.*\[(.*)\]$',line)
            if test[0] == 'edit':
                test3 = re.findall('(^.*)\[.*\]',line)
                states.append(test3[0])
                state = test3[0]
                count2+=1
                continue
        if re.search('\*',line) != None:
            dct['Carrollton'] = state
            count+=1
            continue
        elif re.search('^(.*?) \(.*$',line) != None:
            new = re.findall('^(.*?) \(.*$',line)
            dct[new[0]]=state
            count+=1
            continue
        elif re.search('^(.*)\[.*$',line) != None:
            new = re.findall('^(.*)\[.*$',line)
            dct[new[0]]=state
            count+=1
            continue
        elif re.search('^(.+,.+)$', line) != None:
            new = re.findall('^(.+,.+)$', line)
            dct[new[0]]=state
            count+=1
            continue
        elif re.search('^(.+:)', line) != None: 
            new = re.findall('^(.+:)', line)
            dct[new[0]]=state
            count+=1
            continue
    return dct
def get_list_of_university_towns():
    with open('university_towns.txt', 'r') as uni:
        dct = clean_text(uni)
    df = pd.DataFrame(list(dct.items()), columns = ['RegionName','State'])
    df = df[['State','RegionName']]
    df['RegionName'] = df['RegionName'].str.replace('_[0-9]+$','')
    return df
get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,South Carolina,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
...,...,...
484,Wisconsin,River Falls
485,Wisconsin,Stevens Point
486,Wisconsin,Waukesha
487,Wisconsin,Whitewater


In [5]:
gdp = pd.read_excel('gdplev.xls', skiprows=[0,1,2,3,4,5,6,7],header = None, footer = None)
gdp.drop(gdp.columns[3], axis=1,inplace=True)
gdpA = gdp[[0,1,2]].dropna()
gdpQ = gdp[[4,5,6]].set_index(4)
gdp = gdpQ[212:]
def get_recession_start():
    for i in range(2,len(gdp)):
        if (gdp.iloc[i,1] < gdp.iloc[i-1,1]) & (gdp.iloc[i-1,1] < gdp.iloc[i-2,1]):
            start = gdp.iloc[i-1].name
            break
    return str(start)
get_recession_start()

'2008q3'

In [6]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    for i in range(2,len(gdp)):
        if (gdp.iloc[i,1] < gdp.iloc[i-1,1]) & (gdp.iloc[i-1,1] < gdp.iloc[i-2,1]):
            start = gdp.iloc[i-1].name
            break
    for j in range(i-1,len(gdp)):
        if (gdp.iloc[j,1] > gdp.iloc[j-1,1]) & (gdp.iloc[j-1,1] > gdp.iloc[j-2,1]):
            end = gdp.iloc[j].name
            break
    return end
get_recession_end()

'2009q4'

In [7]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    min=0
    for i in range(2,len(gdp)):
        if (gdp.iloc[i,1] < gdp.iloc[i-1,1]) & (gdp.iloc[i-1,1] < gdp.iloc[i-2,1]):
            start = gdp.iloc[i-1].name
            break
    for j in range(i-1,len(gdp)):
        if (gdp.iloc[j,1] > gdp.iloc[j-1,1]) & (gdp.iloc[j-1,1] > gdp.iloc[j-2,1]):
            end = gdp.iloc[j].name
            break
    bottom = gdp.iloc[i-1:j,1].idxmin()
    return bottom
get_recession_bottom()

'2009q2'

In [8]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    data = pd.read_csv('City_Zhvi_AllHomes.csv')
    lst = list()
    dat1 = data.iloc[:,[2,1]]
    dat2 = data.iloc[:,51:251]
    dat2.columns = pd.to_datetime(dat2.columns)
    dat2 = dat2.T.resample('Q').mean().T
    new, year, month, ct1, cty, ctm = '', '20', '-', 0, 0, -2
    for i in range(0,67):
        if ct1 == 4: ct1 = 0
        if ctm+2 == 12: cty +=1
        if ctm > 8: ctm = -2
        ct1 += 1
        ctm += 3
        new = year + "{0:0=2d}".format(cty) + 'q' + str(ct1)
        lst.append(new)
    dat2.columns = lst
    data = dat1.join(dat2)
    data['State'] = data['State'].map(states)
    return data
convert_housing_data_to_quarters()

,State,RegionName,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
0,New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
1,California,Los Angeles,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0
2,Illinois,Chicago,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0
3,Pennsylvania,Philadelphia,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,128700.0
4,Arizona,Phoenix,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,195200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10725,Wisconsin,Town of Wrightstown,101766.666667,105400.000000,111366.666667,114866.666667,125966.666667,129900.000000,129900.000000,129433.333333,...,144866.666667,146866.666667,149233.333333,148666.666667,149333.333333,149866.666667,149933.333333,149833.333333,151266.666667,155000.0
10726,New York,Urbana,79200.000000,81666.666667,91700.000000,98366.666667,94866.666667,98533.333333,102966.666667,98033.333333,...,132133.333333,137033.333333,140066.666667,141700.000000,137866.666667,136466.666667,136166.666667,138966.666667,144200.000000,143000.0
10727,Wisconsin,New Denmark,114566.666667,119266.666667,126066.666667,131966.666667,143800.000000,146966.666667,148366.666667,149166.666667,...,174566.666667,181166.666667,186166.666667,187600.000000,188666.666667,188433.333333,188933.333333,191066.666667,192833.333333,197600.0
10728,California,Angels,151000.000000,155900.000000,158100.000000,167466.666667,176833.333333,183766.666667,190233.333333,184566.666667,...,244466.666667,254066.666667,259933.333333,260100.000000,250633.333333,263500.000000,279500.000000,276533.333333,271600.000000,269950.0


In [9]:
def run_ttest():
    from warnings import filterwarnings
    filterwarnings('ignore')
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    hdf = convert_housing_data_to_quarters()
    unis = get_list_of_university_towns()
    start = get_recession_start()
    bstart = hdf.columns[hdf.columns.get_loc(start)-1]
    bottom = get_recession_bottom()
    df = pd.merge(hdf.reset_index(drop=True),unis,on = unis.columns.tolist(),indicator='_flag',how = 'outer')
    df1 = df[df['_flag']=='both']
    df2 = df[df['_flag']!='both']
    df1.dropna(subset=df1.columns.tolist()[hdf.columns.get_loc(start)-1:hdf.columns.get_loc(bottom)], how = 'all', inplace=True)
    df2.dropna(subset=df2.columns.tolist()[hdf.columns.get_loc(start)-1:hdf.columns.get_loc(bottom)], how = 'all', inplace=True)
    pr1 = df1[bstart].div(df1[bottom]).dropna()
    pr2 = df2[bstart].div(df2[bottom]).dropna()
    [s,p] = ttest_ind(pr1,pr2)
    if p < 0.01:
        different = True
    else: different = False
    if s < -0.01:
        better = 'university town'
    elif s > 0.01:
        better = 'non-university town'
    return different,p,better
run_ttest()

(True, 0.0026083039193693608, 'university town')